In [8]:
import numpy as np  # import numpy for operations on vectorized arrays
# import bokeh palette for color coding on graphs
from bokeh.palettes import Category20_20 as palette
# import bokeh components for plotting
from bokeh.plotting import figure,  show
from bokeh.models import ColumnDataSource, Range1d, LabelSet, Label
from bokeh.io import push_notebook, show, output_notebook
# import odeint to integrate system of ODE
from scipy.integrate import odeint
import csd  # import CSD.py file with functions and constants
output_notebook()                          # enable bokeh to Jupyter output extnsion

# ---------------- import and prepare element data ---------------


ELEMENT_NAME = 'Pb'                   # specify element to study by name
# convert element data to a dictionary with charge states as iteger keys
Elem =  csd.get_element_data(ELEMENT_NAME)


# --------------import predefined constants---------------

IP = csd.CONST["Ry"]        # ionization potential of rest gas

# ----------------define simulation variables---------------

E_e = 5025       # electron energy eV
P_VAC = 1E-10    # vacuum pressure mbar
T_ion = 100           # ion temperature in eV
J_e = 1000            # A/cm2

ch_states = np.linspace(0, len(Elem), len(Elem)+1)   # define charge states

# ---------define initial conditions and time frame---------------

initial_CSD = np.zeros(len(ch_states)) # all zeros CSD
initial_CSD[0] = 1 # Pulsed injection of neutral gas

time = np.logspace(-6, 0, num=1000)          # generate  log linear time range

# ----- -----define time independent reaction rates----------------------

rates = csd.get_reaction_rates(elem=Elem, j_e=J_e, e_e=E_e,
                               t_ion=T_ion, p_vac=P_VAC, ip=IP, ch_states=ch_states)

# -----------------solve system of ODEs--------------------

solution = odeint(csd.csd_evolution, initial_CSD, time,
                  args=rates)  # integrate ODE system
CSD_plot = csd.csd_base_figure()  # instantinate default CSD figure

# --------------populate CSD figure and legend------------------

colors = [csd.color_picker(len(ch_states), i, palette)
          for i in range(len(ch_states))]

for i in range(0, len(ch_states), 1):

    CSD_plot.line(time, solution[:, i], color=colors[i],  line_width=3,
                  muted_alpha=0.2, muted_color=colors[i], legend_label=element_name+str(i)+'+')
    peak_label = Label(x=time[np.argmax(solution[:, i])], y=max(solution[:, i])+0.01, text=str(i)+'+',
                       text_color=colors[i])
    CSD_plot.add_layout(peak_label)
    CSD_plot.title.text = element_name+',  Eₑ = ' + \
        str(round(E_e/1000, 2))+' keV, '+'Jₑ=' + \
        str(round(J_e, 0)) + ' A/cm², '+'Pᵥ='+str(P_vac)+' mbar '
    CSD_plot.title.text_font_style = "normal"

show(CSD_plot)  # show results in the notebook

Loading BokehJS ...